In [22]:
import struct
import math
import pyarrow as pa
import pyarrow.parquet as pq
import sys, os
from pathlib import Path
import dask.dataframe as dd
from time import sleep
import pandas as pd
pd.options.display.max_columns = 2000
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 10000)

In [2]:
parquet_folder="/p/gpfs1/haridev/recorder_logs/lbann_jag_32/_parquet"

In [3]:
ddf = dd.read_parquet("{}/*.parquet".format(parquet_folder), engine="pyarrow")

In [4]:
ddf.head()

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
0,0,0.005714,2.339396,MPI_Comm_dup,0,3,MPI_COMM_WORLD,0-0,0,,,,,,,
1,0,2.338808,2.338814,open,-1,3,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,557250,384,,,,,,,
2,0,2.338852,2.338855,open,53,2,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,557058,,,,,,,,
3,0,2.338871,2.338876,ftruncate,0,2,53,2691072,,,,,,,,
4,0,2.338878,2.338881,mmap,0,6,%p,2691072,3,1,53,0,,,,


In [5]:
ddf_rank_0 = ddf[ddf["rank"] ==0]

In [6]:
ddf_rank_0.head()

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
0,0,0.005714,2.339396,MPI_Comm_dup,0,3,MPI_COMM_WORLD,0-0,0,,,,,,,
1,0,2.338808,2.338814,open,-1,3,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,557250,384,,,,,,,
2,0,2.338852,2.338855,open,53,2,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,557058,,,,,,,,
3,0,2.338871,2.338876,ftruncate,0,2,53,2691072,,,,,,,,
4,0,2.338878,2.338881,mmap,0,6,%p,2691072,3,1,53,0,,,,


In [7]:
ddf_rank_0_open = ddf_rank_0[(ddf_rank_0["func_id"] == "open") &
                             (ddf_rank_0["arg_count"] == 2) &
                             (~ddf_rank_0["res"].isin([-1,0]))]

In [16]:
filenames = ddf[(ddf["args_1"].str.contains("/p/gpfs1/"))]['args_1'].unique().compute()

In [23]:
print(filenames)

0    /p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext
1                                                      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy
Name: args_1, dtype: object


In [74]:
selected_rows = []
for rank in range(128):
    selected_rows.append(ddf[(ddf["args_1"].isin(filenames)) & (ddf["rank"] ==rank)])

In [76]:
selected_rank = 0

In [77]:
selected_rows[selected_rank].compute()

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
10089,0,20.513746,20.517872,open64,83,2,/p/gpfs1/haridev/software/lbann/applications/physics/ICF/20210824_103242_jag_wae/experiment.prototext,0,,,,,,,,
17393,0,20.715036,20.737030,fopen64,83,2,/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy,r,,,,,,,,
17395,0,20.737125,20.737143,fopen,83,2,/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy,rb,,,,,,,,


In [79]:
interested_functions = selected_rows[selected_rank]['func_id'].unique().compute()

In [80]:
interested_res = selected_rows[selected_rank]['res'].unique().compute()

In [81]:
operations_expected = selected_rows[selected_rank]['args_2'].unique().compute()

In [82]:
posix_open = ["creat",        "creat64",      "open",         "open64"]
posix_read = ["read", "pread", "pread64", "readv"]
posix_write = ["write", "pwrite", "pwrite64","writev"]
stdio_open = ["fopen", "fopen64"]
stdio_read = ["fread"]
stdio_write = ["fwrite"]
MPI_open = ["PMPI_File_open"]
MPI_read = ["PMPI_File_iread_at", "PMPI_File_iread", "PMPI_File_iread_shared", "PMPI_File_read_all_begin",     "PMPI_File_read_all",       "PMPI_File_read_at_all",
    "PMPI_File_read_at_all_begin",  "PMPI_File_read_at",        "PMPI_File_read",
    "PMPI_File_read_ordered_begin", "PMPI_File_read_ordered",   "PMPI_File_read_shared"]
MPI_write = ["PMPI_File_iwrite_at", "PMPI_File_iwrite",             "PMPI_File_iwrite_shared","PMPI_File_write_all_begin",
    "PMPI_File_write_all",          "PMPI_File_write_at_all_begin", "PMPI_File_write_at_all",
    "PMPI_File_write_at",           "PMPI_File_write",          "PMPI_File_write_ordered_begin",
    "PMPI_File_write_ordered",      "PMPI_File_write_shared"]
HDF5_open = ["H5Fcreate",            "H5Fopen"]
HDF5_read = ["H5Dread"]
HDF5_read = ["H5Dwrite"]

In [83]:
interested_functions

0     open64
1    fopen64
2      fopen
Name: func_id, dtype: object

In [84]:
interested_res = interested_res.apply(str)
print(interested_res)

0    83
Name: res, dtype: object


In [85]:
operations_expected_str = set()
for ops in operations_expected:
    if "r" in ops or "0" in ops:
        operations_expected_str.add("read")
    elif "w" in ops or "1" in ops:
        operations_expected_str.add("write")
print(operations_expected_str)

{'read'}


In [86]:
read_funcs = posix_read
read_funcs.extend(stdio_read)
print(read_funcs)

['read', 'pread', 'pread64', 'readv', 'fread']


In [90]:
operation_rows = ddf[(ddf["rank"] == selected_rank) & (ddf["func_id"].isin(stdio_read)) & (ddf['args_4'].isin(interested_res))]

In [91]:
operation_rows.compute()

,rank,tstart,tend,func_id,res,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
17396,0,20.737194,20.796473,fread,0,4,%p,1,11,83,,,,,,
17397,0,21.974136,28.470678,fread,0,4,%p,1,13405611264,83,,,,,,
